# Before you start :
   - These exercises are related to the Pivot table and correlation lessons.
   - Keep in mind that you need to use some of the functions you learned in the previous lessons.
   - All datasets are provided in the `your-code` folder of this lab.
   - Elaborate your codes and outputs as much as you can.
   - Try your best to answer the questions and complete the tasks and most importantly enjoy the process!!!

#### Import all the libraries that are necessary.

In [100]:
# your answer here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Challenge 1

#### Open the ` Employee.csv` file and name your dataset `employee`.

In [101]:
# your answer here
employee = pd.read_csv('data/Employee.csv')
employee

,Name,Department,Education,Gender,Title,Years,Salary
0,Jose,IT,Bachelor,M,analyst,1,35
1,Maria,IT,Master,F,analyst,2,30
2,David,HR,Master,M,analyst,2,30
3,Sonia,HR,Bachelor,F,analyst,4,35
4,Samuel,Sales,Master,M,associate,3,55
5,Eva,Sales,Bachelor,F,associate,2,55
6,Carlos,IT,Master,M,VP,8,70
7,Pedro,IT,Phd,M,associate,7,60
8,Ana,HR,Master,F,VP,8,70


#### What's the mean salary by department?
Hint : There are 2 ways to do it - one way is with and the other way is without pivot tables. Do it both ways. 


In [102]:
# Method 1
# your answer here
employee.groupby('Department').agg({'Salary': 'mean'})

,Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


In [103]:
# Method 2
# your answer here

#no need to pass aggfunc = 'mean' as the default is already the mean
employee.pivot_table(index = 'Department', values = 'Salary')

,Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


#### We want now to group the data by more than one column. Use Pandas pivot table function to select the mean salary by department and title.

In [104]:
# your answer here
employee.pivot_table(index = ['Department', 'Title'], values = 'Salary')

Salary
Department Title            
HR         VP           70.0
           analyst      32.5
IT         VP           70.0
           analyst      32.5
           associate    60.0
Sales      associate    55.0

#### We want to display the number of years of work experience and the salary for each Title. Add Years as a column in your pivot table.
*Hint: Use Pandas doc(see References in README.md) to explore the pivot_table() arguments*

In [105]:
# your answer here

# Ok, this actually looks super cool
employee.pivot_table(index = ['Department', 'Title'], values = 'Salary', columns = 'Years', 
                     fill_value = '-')

Years                  1   2   3   4   7   8
Department Title                            
HR         VP          -   -   -   -   -  70
           analyst     -  30   -  35   -   -
IT         VP          -   -   -   -   -  70
           analyst    35  30   -   -   -   -
           associate   -   -   -   -  60   -
Sales      associate   -  55  55   -   -   -

#### We want to know the information about the mean salary and number of employees in each department and for each job title.

In [107]:
# your answer here
employee.pivot_table(index = ['Department', 'Title'], values = ['Salary', 'Gender'], aggfunc = 
                     {'Salary': 'mean', 'Gender': 'count'}).rename(columns = 
                                                                   {'Gender': '# Employees'})

# Employees  Salary
Department Title                         
HR         VP                   1    70.0
           analyst              2    32.5
IT         VP                   1    70.0
           analyst              2    32.5
           associate            1    60.0
Sales      associate            2    55.0

#### We want to know for each department and for each job title the Salary's median, minimum, maximum and standard deviation. Comment your results.

In [110]:
# your answer here

'''
Well, for HR-VP / IT-VP / IT-associate there is only one person there, that's why the values are
always the same and there is no STD.
We can confirm that by looking at the other pivot table above, as well, in the '# Employees' col.
'''
almost_a_dot_describe = employee.pivot_table(index = ['Department', 'Title'], values = 'Salary', 
                     aggfunc = ['median', 'min', 'max', 'std'])

almost_a_dot_describe

median    min    max       std
                     Salary Salary Salary    Salary
Department Title                                   
HR         VP          70.0     70     70       NaN
           analyst     32.5     30     35  3.535534
IT         VP          70.0     70     70       NaN
           analyst     32.5     30     35  3.535534
           associate   60.0     60     60       NaN
Sales      associate   55.0     55     55  0.000000

#### Based on your comments, fill the missing values with an appropriate value.

In [111]:
# your answer here

# Tried filling above directly in the same line like before, with the 'fill_value' arg, but this
# time it didn't work, for some reason
almost_a_dot_describe.fillna(0)

median    min    max       std
                     Salary Salary Salary    Salary
Department Title                                   
HR         VP          70.0     70     70  0.000000
           analyst     32.5     30     35  3.535534
IT         VP          70.0     70     70  0.000000
           analyst     32.5     30     35  3.535534
           associate   60.0     60     60  0.000000
Sales      associate   55.0     55     55  0.000000

#### The stake holders want to know for each department the number of employees and how much money is spent on salaries. Could you provide that information? 

In [112]:
# your answer here
employee.pivot_table(index = 'Department', values = ['Salary', 'Gender'], 
                     aggfunc = {'Salary': 'sum', 'Gender': 'count'})

,Gender,Salary
Department,,
HR,3,135
IT,4,195
Sales,2,110


#### For each Department and Title, we want to know the total years of work experience and the mean salary.

In [113]:
# your answer here
employee.pivot_table(index = ['Department', 'Title'], values = ['Salary', 'Years'], 
                     aggfunc = {'Salary': 'mean', 'Years': 'sum'})

Salary  Years
Department Title                   
HR         VP           70.0      8
           analyst      32.5      6
IT         VP           70.0      8
           analyst      32.5      3
           associate    60.0      7
Sales      associate    55.0      5

### Bonus

#### We now want to compute the mean salary after removing the maxima for each Department. Create the appropriate pivot table.

Hint: Write a custom function.

In [ ]:
# your answer here


# Challenge 2

#### Open the ` Fitbit.csv` file and name your dataset `fitbit`.

In [ ]:
# your answer here


#### Explore you dataset in terms of data types and descriptive statistics.
Hint: Use Pandas functions from previous lectures.

In [ ]:
# your answer here


In [ ]:
# your answer here


#### You suspect that there must be a linear relationship between the Minutes Very Active and the Steps. Compute the correlation between these variables.

In [ ]:
# your answer here


#### Use matplotlib to visually plot Minutes Very active vs Steps. 

##### Hint : import matplotlib.pyplot as plt (See previous lessons)#####
- Use plt.scatter(x=df['col_name_1'], y=df['col_name_2'])

In [ ]:
# your answer here


#### What can you say about Minute Very Active and Steps? Write a comment below

In [ ]:
# your answer here


#### We also suspect that there must be a linear relationship between the Minutes Sedentary and the Steps. Compute the correlation between these variables.

In [ ]:
# your answer here


#### Use matplotlib to visually plot Minutes Sedentary vs Steps. Based on the results of the computed correlation and the plot. What can you say about these 2 variables?

In [ ]:
# your answer here


#### We also suspect that there must be a linear relationship between the MinutesOfSleep and the Steps. Compute the correlation between these variables.

In [ ]:
# your answer here


#### Use matplotlib to visually plot MinutesOfSleep vs Steps. Based on the results of the computed correlation and the plot. What can you say about these 2 variables?

In [ ]:
# your answer here


#### Select a subset of your dataset with the columns below and compute the correlation matrix. Save the results in the variable `cor_fit`
column= ['Calorie burned', 'Steps','Floors','Minutes Sedentary','Minutes Very Active', 'MinutesOfSleep']

In [ ]:
# your answer here


#### Based on the result of the correlation matrix. What are the highly correlated features with the variable Steps?
*Hint: From the results above select the variable Steps and order your results in descending order (use Pandas sort_values function)*

In [ ]:
# your answer here


# Challenge 3

#### Open the `Time_Grades.csv` file and name your dataset `time_grades`.

In [ ]:
# your answer here


#### Show visually the frequency distribution of the time_grades dataset.

In [ ]:
# your answer here


#### Is there a correlation between study time and Grade? Use both Pearson and Spearman correlation and comment your results.

In [ ]:
# your answer here


#### Use matplotlib to visually plot Study time vs Grade. Based on the results of the computed correlation and the plot. What can you say about these 2 variables?

In [ ]:
# your answer here


#### Could you explain the difference between Pearson and Spearman correlation? (see References in README.md) 

In [ ]:
# your answer here
